In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Load data

In [3]:
import sys
sys.path.append("runormas/")

In [4]:
from modules.utils import get_all_files_from_dir

In [5]:
from score import score, regularize
import re

In [131]:
from collections import defaultdict
import os


y_true = defaultdict(dict)
y_pred = defaultdict(dict)
errors = []
total_named = 0
total_generic = 0
error_named = 0
error_generic = 0
files = get_all_files_from_dir("reference/")
for fn in files:
    n_key = os.path.basename(fn)
    if "named" in fn:
        part = "named"
    else:
        part =  "generic"
    key = n_key[:-5]
    y_true[part][key] = []
    y_pred[part][key] = []
    with open(fn) as file:
        gt = regularize(file.read().strip())
        
        if "named" in fn:
            y_true[part][key] = gt.split("\n")
            total_named += len(y_true[part][key])
        else:
            gt = re.sub(" ", "", gt)
            y_true[part][key] = gt.lower().split("\n")
            total_generic += len(y_true[part][key])

    fn = os.path.join("test_pred/v14_130k_finetune_fixed", part, n_key)
    if os.path.exists(fn):
        with open(fn) as file:
            gt = regularize(file.read().strip())
            if "named" in fn:
                y_pred[part][key] = gt.split("\n")
            else:
                gt = re.sub(" ", "", gt)
                y_pred[part][key] = gt.lower().split("\n")
            for idx, yt in enumerate(y_true[part][key]):
                yp = y_pred[part][key][idx]
                if yp != yt:
                    if "named" in fn:
                        error_named += 1
                    else:
                        error_generic += 1
                    errors.append({
                        "part": part,
                        "key": key,
                        "type": "not_matched",
                        "y_true": yt,
                        "y_pred": yp,
                        "idx": idx
                    })
    else:
        for idx, yt in enumerate(y_true[part][key]):
            if "named" in fn:
                error_named += 1
            else:
                error_generic += 1
            errors.append({"part": part, "key": key, "type": "no_file", "y_true": yt, "idx": idx})

In [132]:
1 - error_generic / total_generic, 1 - error_named / total_named

(0.9645180366646955, 0.9534031413612566)

In [ ]:
files = get_all_files_from_dir("baseline/")
baseline = defaultdict(dict)

for fn in files:
    n_key = os.path.basename(fn)
    if "named" in fn:
        part = "named"
    else:
        part =  "generic"
    key = n_key[:-5]
    baseline[part][key] = []
    with open(fn) as file:
        gt = regularize(file.read().strip())
        if "named" in fn:
            baseline[part][key] = gt.split("\n")
        else:
            gt = re.sub(" ", "", gt)
            baseline[part][key] = gt.lower().split("\n")

In [ ]:
from modules.data.read import DataReader

reader = DataReader(
    path="data/public_test/",
    output_dir="data/public_test_v13/",
    tokenizer_name="sberbank-ai/rugpt3xl",
    part="test",
    local_rank=0,
    word_size=1,
    window_size=40
)

In [ ]:
reader.prc(is_save=False)

In [63]:
len(errors)

856

In [64]:
# score(predict_dir="test_pred/v14_130k_finetune_fixed", true_dir="reference/")

In [177]:
wc_named = 0
wc_generic = 0
wc_errors = []
not_ws_errors = []
for err in errors:
    line_ann = reader.anns[err["part"]][err["key"]][err["idx"]].strip().split()
    text = reader.texts[err["part"]][err["key"]]
    start, stop = list(map(int, line_ann))
    norm = regularize(text[start:stop])
    if "named" != err["part"]:
        norm = re.sub(" ", "", norm)
    if len(norm.split()) != len(err["y_true"].split()):
        err["norm"] = text[start:stop]
        wc_errors.append(err)
        # print(norm, err["y_true"], err["y_pred"], sep=" | ")
        if "named" == err["part"]:
            wc_named += 1
            if norm == err["y_true"]:
                wc_named += 1
                
        else:
            wc_generic += 1
            if norm == err["y_true"]:
                wc_generic += 1
    else:
        err["norm"] = text[start:stop]
        not_ws_errors.append(err)

In [178]:
wc_named, wc_generic

(170, 1)

In [180]:
# wc_errors

In [136]:
len(wc_errors)

171

In [181]:
is_titled_errors = []
not_is_titled_errors = []
for err in not_ws_errors:
    ytt = sum([x.istitle() for x in err["y_true"].split()])
    ypt = sum([x.istitle() for x in err["norm"].split()])
    if ytt != ypt:
        is_titled_errors.append(err)
        # print(err["norm"], err["y_true"], err["y_pred"], sep=" | ")
    else:
        not_is_titled_errors.append(err)

In [184]:
# is_titled_errors

In [182]:
len(not_ws_errors), len(not_is_titled_errors)

(663, 513)

In [139]:
not_xa_errors = []
xa_errors = []
for err in not_is_titled_errors:
    if "\xa0" in err["norm"] and "\xa0" not in err["y_true"]:
        xa_errors.append(err)
    else:
        not_xa_errors.append(err)

In [140]:
len(not_is_titled_errors), len(not_xa_errors)

(513, 488)

In [141]:
from string import punctuation

In [142]:
punctuation = set(punctuation + "«»")

In [143]:
not_punct_err = []
punct_err = []
for err in not_xa_errors:
    is_err = False
    for sym in err["norm"]:
        if sym in punctuation and sym not in err["y_true"]:
            is_err = True
            break
    for sym in err["y_true"]:
        if sym in punctuation and sym not in err["norm"]:
            is_err = True
            break
    
    if  err["norm"][-1] in punctuation and err["y_true"][-1] not in punctuation:
        punct_err.append(err)
    elif err["norm"][0] in punctuation and err["y_true"][0] not in punctuation:
        punct_err.append(err)
    elif "–" in err["y_true"] and "–" not in err["norm"]:
        punct_err.append(err)
    elif "*" not in err["y_true"] and "*" in err["norm"]:
        punct_err.append(err)
    elif is_err:
        punct_err.append(err)
    else:
        not_punct_err.append(err)

In [144]:
len(not_xa_errors), len(not_punct_err)

(488, 446)

In [145]:
ws_errors = []
not_ws_errors = []
for err in not_punct_err:
    yt = " ".join(err["y_true"].split())
    yp = " ".join(err["y_pred"].split())
    if yt == yp:
        ws_errors.append(err)
    else:
        not_ws_errors.append(err)

In [146]:
len(not_ws_errors), len(ws_errors)

(446, 0)

In [153]:
word_start_errors = []
not_word_start_errors = []
for err in not_ws_errors:
    is_err = False
    for yt, yp in zip(err["y_true"].split(), err["norm"].split()):
        if yt[0] != yp[0]:
            is_err = True
            break
    if is_err:
        word_start_errors.append(err)
    else:
        not_word_start_errors.append(err)

In [154]:
len(not_ws_errors), len(not_word_start_errors)

(446, 415)

In [162]:
1 - len([x for x in not_word_start_errors if x["part"] == "generic"]) / total_generic

0.9767001774098166

In [161]:
1 - len([x for x in not_word_start_errors if x["part"] == "named"]) / total_named

0.980977312390925